In [3]:
import rospy
import os
from move_base_msgs.msg import MoveBaseActionFeedback
from move_base_msgs.msg import MoveBaseActionGoal
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from actionlib_msgs.msg import GoalID
from actionlib_msgs.msg import GoalStatusArray
import sys, tty
from sensor_msgs.msg import LaserScan
from IPython.core.display import display, HTML
from IPython.display import display,clear_output
from ipywidgets import AppLayout, Button, Layout, ButtonStyle, GridBox, VBox, HBox , widgets , interact, interactive, fixed, interact_manual, interactive_output
import time
import matplotlib.pyplot as plt 
from move_base_msgs.msg import MoveBaseActionResult

#from jupyros import ros3d

In [ ]:
%matplotlib widget
def clbk_laser(msg):
        min_fright = min(min(msg.ranges[144:287]), 5)
        min_front =  min(min(msg.ranges[288:431]), 5)
        min_fleft =  min(min(msg.ranges[432:575]), 5)
        global vel_pub, my_vel, min_assfront
        if(min_front < 1):
            [my_vel.linear.x, my_vel.angular.z] =[0 , 0]
            vel_pub.publish(my_vel)
            

        with min_assfront: 
            print('\r                                          ', end='', flush=True)
            print(f'\r{min_front}', end='', flush=True)
            
        return min_fright, min_front, min_fleft

def plt_(a):
    %matplotlib widget
    global plot_laser
    if(plot_laser == None):
        plt.ion()
        plt.plot(0)
    else: 
        plt.ion()
        plt.plot(plot_laser)
    

    
    
    
def frw(a):
    global my_vel, vel_pub,state
    laser = rospy.wait_for_message("/scan", LaserScan)
    [min_fright, min_front, min_fleft] = clbk_laser(laser)
    [my_vel.linear.x, my_vel.angular.z] =[1 , 0]
    vel_pub.publish(my_vel)
    if min_front < 1:
        [my_vel.linear.x, my_vel.angular.z] =[0 , 0]
        vel_pub.publish(my_vel)
        with state:
            print('\r                           ', end='', flush=True)
            print(f'\rAssisted modality', end='', flush=True)
        take_action(min_fright, min_front, min_fleft)
                        
    else:    
        with state:
            print('\r                                                ', end='', flush=True)
            print(f'\rManual modality', end='', flush=True)
        [my_vel.linear.x, my_vel.angular.z] =[1 , 0]
        vel_pub.publish(my_vel)
    
def be(a):
    global my_vel, vel_pub,state
    laser = rospy.wait_for_message("/scan", LaserScan)
    [min_fright, min_front, min_fleft] = clbk_laser(laser)

    if min_front < 1 or min_fleft < 1 or min_fright < 1:
        with state:
            print('\r                                                      ', end='', flush=True)
            print(f'\rAssisted modality', end='', flush=True)
        take_action(min_fright, min_front, min_fleft)

                        
    else:    
        [my_vel.linear.x, my_vel.angular.z] =[-1 , 0]
        vel_pub.publish(my_vel)
        with state:
            print('\r                                            ', end='', flush=True)
            print(f'\rManual modality', end='', flush=True)

    
def le(a):
    global my_vel, vel_pub,state
    laser = rospy.wait_for_message("/scan", LaserScan)
    [min_fright, min_front, min_fleft] = clbk_laser(laser)

    if min_front < 1 or min_fleft < 1 or min_fright < 1:
        with state:
            print('\r                                              ', end='', flush=True)
            print(f'\rAssisted modality', end='', flush=True)
        take_action(min_fright, min_front, min_fleft)
                        
    else:    
        with state:
            print('\r                                           ', end='', flush=True)
            print(f'\rManual modality', end='', flush=True)
        [my_vel.linear.x, my_vel.angular.z] =[0 , 2]
        vel_pub.publish(my_vel)
    
    
def ri(a):
    global my_vel,vel_pub,state
    laser = rospy.wait_for_message("/scan", LaserScan)
    [min_fright, min_front, min_fleft] = clbk_laser(laser)
 
    if min_front < 1 or min_fleft < 1 or min_fright < 1:
        with state:
            print('\r                                       ', end='', flush=True)
            print(f'\rAssisted modality', end='', flush=True)
        take_action(min_fright, min_front, min_fleft)
        
                        
    else:    
        with state:
            print('\r                                       ', end='', flush=True)
            print(f'\rManual modality', end='', flush=True)
        [my_vel.linear.x, my_vel.angular.z] =[0, -2]
        vel_pub.publish(my_vel)

        
def exm(a):
    global exit,amm_bol 
    amm_bol = False
    exit = False
    [my_vel.linear.x, my_vel.angular.z] =[0 , 0]
    vel_pub.publish(my_vel)
    into()
    
    
    
def assisted_manual_show():
    
        global vel_pub,my_vel, plot_laser
        clear_output(wait = True)
        plot_laser = []
        vel_pub = rospy.Publisher("cmd_vel", Twist, queue_size= 50)
        #laser = rospy.wait_for_message("/scan", LaserScan)
        
        my_vel = Twist()
        my_vel.linear.y = 0
        my_vel.linear.z = 0
        my_vel.angular.x = 0
        my_vel.angular.y= 0
        
        laser = rospy.wait_for_message("/scan", LaserScan)
            
        plot_laser.append(laser.ranges[435:440])

        if (len(plot_laser) > 100):
            for i in range(20):
                plot_laser.pop(i)
        global min_assfront
        min_assfront = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 

    
        
        global min_fright, min_front, min_fleft
        [min_fright, min_front, min_fleft] = clbk_laser(laser)
        
    

        
        
        #------------------------------------------------------------
        display(HTML('<h1 style = "position:relative; margin-left:370px;">Assisted Motion Planner</h1>'))
        print('\n')
        print('\n')
        state_description = ''

        forw = Button(description='^',layout=Layout(width='auto', grid_area='forw'), style=ButtonStyle(button_color='lightblue'))
        lef = Button(description='<', layout=Layout(width='auto', grid_area='lef'), style=ButtonStyle(button_color='lightblue'))
        rig = Button(description='>', layout=Layout(width='auto', grid_area='rig'), style=ButtonStyle(button_color='lightblue'))
        beh = Button(description='v', layout=Layout(width='auto', grid_area='beh'), style=ButtonStyle(button_color='lightblue'))

        field = GridBox(children=[forw,lef,rig,beh], layout=Layout(width='auto',grid_template_rows='auto auto auto', grid_template_columns='auto auto auto', grid_template_areas='''
        " . forw . " 
        "lef . rig "
        " . beh . "
        ''')
        )
        #------------------------------------------------------------
        display(field)
        print('\n')
        display(HTML('<h3 style = "position:relative; margin-left:0px;">Closest obstacle distance:</h3>'))
        print('\n')
        
        display(min_assfront)
        display(HTML('<h3 style = "position:relative; margin-left:0px;">State:</h3>'))
        print('\n')
        global state
        state = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
        display(state)
        print('\n')
        
        plt_pos = Button(description='Plot position', layout=Layout(width='100', margin= '0px 0px 0px 700px'), style=ButtonStyle(button_color='orange'))
        display(plt_pos)
        
        exitm = Button(description='Exit', layout=Layout(width='100', margin= '0px 0px 0px 700px'), style=ButtonStyle(button_color='red'))
        display(exitm)
        state_description = ''
       
        

        #------------------------------------------------------------
        
        
        exitm.on_click(exm)
        plt_pos.on_click(plt_)
        forw.on_click(frw)
        lef.on_click(le)
        rig.on_click(ri)
        beh.on_click(be)
        
        #------------------------------------------------------------
        

assisted_manual_show()
    


Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f2be3935d30>
Traceback (most recent call last):
  File "/usr/lib/python3.8/_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 


In [ ]:
def take_action(min_fright,min_front, min_fleft):
        global my_vel, vel_pub,state
        linear_x = 0
        angular_z = 0

        laser = rospy.wait_for_message("/scan", LaserScan)
        [min_fright, min_front, min_fleft] = clbk_laser(laser)
        
        #print(min_fright,min_front, min_fleft)
        my_vel.linear.y = 0
        my_vel.linear.z = 0
        my_vel.angular.x = 0
        my_vel.angular.y= 0
   
 
        state_description = 'SWITCH ASSISTED DRIVE'
        
        

        
        if min_front > 1 and min_fleft > 1 and min_fright > 1:
            state_description = 'case 1 - nothing'
            linear_x = 0.6
            angular_z = 0
        elif min_front < 1 and min_fleft > 1 and min_fright > 1:
            state_description = 'case 2 - front'
            linear_x = -2
            angular_z = 0
        elif min_front > 1 and min_fleft > 1 and min_fright < 1:
            state_description = 'case 3 - fright'
            linear_x = 0
            angular_z = 1
        elif min_front > 1 and min_fleft < 1 and min_fright > 1:
            state_description = 'case 4 - fleft'
            linear_x = 0
            angular_z = -1
        elif min_front < 1 and min_fleft > 1 and min_fright < 1:
            state_description = 'case 5 - front and fright'
            linear_x = 0
            angular_z = 1
        elif min_front < 1 and min_fleft < 1 and min_fright > 1:
            state_description = 'case 6 - front and fleft'
            linear_x = 0
            angular_z = -1
        elif min_front < 1 and min_fleft < 1 and min_fright < 1:
            state_description = 'case 7 - front and fleft and fright'
            linear_x = 0
            angular_z = 2
        elif min_front > 1 and min_fleft < 1 and min_fright < 1:
            state_description = 'case 8 - fleft and fright'
            linear_x = 0.3
            angular_z = 0
            

        my_vel.linear.x = linear_x
        my_vel.angular.z = angular_z
        vel_pub.publish(my_vel)
        
        with state:
            print('\r                           ', end='', flush=True)
            print(f'\r{state_description}', end='', flush=True)

        

take_action(0,0,0)

In [6]:
def man(b):
    global manual_bol,exit
    manual_bol = True
    into()
    
def pla(b):
    global plan_bol
    plan_bol = True
    into()
    
def amm(b):
    global amm_bol
    amm_bol=True
    into()
    
def intro():
        
        clear_output(wait = True)
        display(HTML('<h1 style = "position:relative; margin-left:460px;">      MENU</h1>'))
        
        print('\n')
        print('\n')
        
        plan = Button(description='Planning',layout=Layout(width="1000px",height= "100px", grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        manual_motion = Button(description='Manual Motion',layout=Layout(width="1000px",height= "100px", grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        assisted_manual_motion = Button(description=' Assisted Manual Motion',layout=Layout(width="1000px",height= "100px", grid_area='forward'), style=ButtonStyle(button_color='lightblue'))

        boxI = VBox([plan,manual_motion, assisted_manual_motion])
        
        manual_motion.on_click(man)
        plan.on_click(pla)
        assisted_manual_motion.on_click(amm)
        
        display (boxI)


intro()

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

0.0 1.0


IndexError: list index out of range

In [7]:
%matplotlib widget
def into():
        %matplotlib widget
        global manual_bol, exit, plan_bol,amm_bol

             
        if(manual_bol):
            manual_bol = False
            manual_show()

        elif(plan_bol):
            plan_bol = False
            move_base_show()
        
        elif(amm_bol):
            amm_bol = False
            assisted_manual_show()
            
        elif(not exit):
            exit = True
            main()
        else:
            print("No bool True")

    
def main():
        global exit, plan_bol,manual_bol,amm_bol
        global count_target
        
        manual_bol = False
        amm_bol = False
        plan_bol = False
        exit = True
        
        count_target = []
        rospy.init_node('controller')
        intro()
        
 
main()

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

1.0 1.0


In [8]:
def fforward(forward):
    
    [my_vel.linear.x, my_vel.angular.z] =[1 , 0] 
    vel_pub.publish(my_vel)

def fleft(left):
    [my_vel.linear.x, my_vel.angular.z] =[0 , 2]
    vel_pub.publish(my_vel)
    
def fright(right):
    [my_vel.linear.x, my_vel.angular.z] =[0, -2]
    vel_pub.publish(my_vel)
    
def fbehind(behind):
    [my_vel.linear.x, my_vel.angular.z] =[-1 , 0]
    vel_pub.publish(my_vel)

def fquit(quit):
    global exit
    exit = False
    [my_vel.linear.x, my_vel.angular.z] =[0 , 0]
    vel_pub.publish(my_vel)
    into()

    
    

def manual_show():
        
        global my_vel,vel_pub,exit, manual_bol
        vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size = 50)
        my_vel = Twist()
        
        display(HTML('<h1 style = "position:relative; margin-left:340px;">Motion Planner Interface</h1>'))
        print('\n')
        print('\n')


        forward = Button(description='^',layout=Layout(width='auto', grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        left = Button(description='<', layout=Layout(width='auto', grid_area='left'), style=ButtonStyle(button_color='lightblue'))
        right = Button(description='>', layout=Layout(width='auto', grid_area='right'), style=ButtonStyle(button_color='lightblue'))
        behind = Button(description='v', layout=Layout(width='auto', grid_area='behind'), style=ButtonStyle(button_color='lightblue'))
        quit = Button(description='quit', layout=Layout(width='auto', grid_area='quit'), style=ButtonStyle(button_color='red'))

        box = GridBox(children=[forward,left,right,behind,quit], layout=Layout(width='auto',grid_template_rows='auto auto auto', grid_template_columns='auto auto auto', grid_template_areas='''
        " . forward . . ." 
        "left . right . ."
        " . behind . . quit "
        ''')
        )   
        display(box)
        forward.on_click(fforward)
        left.on_click(fleft)
        right.on_click(fright)
        behind.on_click(fbehind)
        quit.on_click(fquit)






manual_show()



GridBox(children=(Button(description='^', layout=Layout(grid_area='forward', width='auto'), style=ButtonStyle(…

In [9]:
#Come uscire fuori?

def manual():
    

    manual_bol = False
    exit = True
    manual_show()
    

In [10]:
%matplotlib widget
def plt_t(a):
    global count_target
    if(count_target == None):
        plt.ion()
        plt.plot(0)
    else: 
        plt.ion()
        plt.plot(count_target)
    
def f(a):
    global x
    try:
        x = float(a.value)
    except:
        print("Please try again")
        
    
def c(a):
    global y
    try:
        y = float(a.value)
    except:
        print("Please try again")
        
def ex(a):
    global exit
    exit = False
    print("Exiting")
    time.sleep(1)
    into()
    
def sub(a):
    global x,y,plan_bol
    try:
        float(x)
        float(y)
        print("Perfect, starting the plan")
        time.sleep(1)
        clear_output(wait=True)
  
        
    except:
        print("Please try again")
    else:
        move_base()
    
        
    
def move_base_show():
         
       global x,y
       x = ""
       y= ""
       count_target = []
       status = None
       result = None
       clear_output(wait=True)
       display(HTML('<h1 style = "position:relative; margin-left:300px;">      PLANNING MOTION INTERFACE</h1>'))
       print('\n')
       print('\n')
       display(HTML('<h3 style = "position:relative; margin-left:0px;"> Insert coordinates, enter and then submit</h3>'))
       
       a = widgets.Text(description='x') 
       b = widgets.Text(description='y') 


    
       coo_box = HBox([widgets.VBox([a, b])])
       display(coo_box)

       a.on_submit(f)
       b.on_submit(c)
        
       submit = Button(description='Submit', layout=Layout(width='100', margin= '0px 0px 0px 700px'), style=ButtonStyle(button_color='lightblue'))
       display(submit)
       exit = Button(description='Exit', layout=Layout(width='100', margin= '0px 0px 50px 700px'), style=ButtonStyle(button_color='red'))
       
       submit.on_click(sub)
       exit.on_click(ex)
       plt_target = Button(description='Plot Target', layout=Layout(width='100', margin= '0px 0px 0px 700px'), style=ButtonStyle(button_color='orange'))
       display(plt_target)
       display(exit)
        
       plt_target.on_click(plt_t)
       
move_base_show()

Button(description='Submit', layout=Layout(margin='0px 0px 0px 700px', width='100'), style=ButtonStyle(button_…

Button(description='Plot Target', layout=Layout(margin='0px 0px 0px 700px', width='100'), style=ButtonStyle(bu…

Button(description='Exit', layout=Layout(margin='0px 0px 50px 700px', width='100'), style=ButtonStyle(button_c…

AttributeError: 'function' object has no attribute 'ion'

In [12]:

def todist(x,y,x_pos,y_pos):
    return ((x-x_pos)**2-(y-y_pos)**2)**0.5

def move_base():

        pub_movebase = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size = 50)
        rate = rospy.Rate(10)
        move = MoveBaseActionGoal()
        move.goal.target_pose.header.frame_id = "map"
        move.goal.target_pose.pose.orientation.w = 1
        position = rospy.wait_for_message("/odom",Odometry)
        
        
        #-------------------------------------
        

        
        
        while(True):
            
                # headings
                clear_output(wait = True)
                display(HTML('<h3 style = "position:relative; margin-left:0px;"> Feedback</h3>'))
                feedbackk = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
                display(feedbackk)
                display(HTML('<h3 style = "position:relative; margin-left:0px;"> Distance to goal</h3>'))
                dtg = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
                display(dtg)
                
                #coordinates
                
                display(HTML('<h3 style = "position:relative; margin-left:0px;">Coordinates:</h3>'))
                

                global x,y
                
                
                
                move.goal.target_pose.pose.position.x = x
                move.goal.target_pose.pose.position.y = y
                print(x,y)
                
                
                pub_movebase.publish(move)
                rate.sleep()
                
                goal_info = None
                
                while(goal_info is None):
                        goal_info = rospy.wait_for_message("move_base/status",GoalStatusArray)
                        rate.sleep()
                status = int(goal_info.status_list[-1].status)  
                #print("status = ",status)
                            
                while(True):
                    
                        
                        goal_info = rospy.wait_for_message("move_base/status",GoalStatusArray)
                        status = int(goal_info.status_list[-1].status)
                        #_id = goal_info.status_list[0].goal_id.id
                        #print(_id)
                                        
                    
                        position = rospy.wait_for_message("/odom",Odometry)
                        x_pos = position.twist.twist.linear.x 
                        y_pos = position.twist.twist.linear.y 
                        
                        
                        
                        #----------------------------------------------
                        if(status == 1):
                            
                            #add Feedback distance, handle it , and print in text box
                        
                            dist = todist(x,y,x_pos,y_pos)
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal Accepted', end='', flush=True)
                            with dtg:
                                print('\r                   ', end='', flush=True)
                                print(f'\r{dist}', end='', flush=True)
                            pass
                        #----------------------------------------------
                        elif(status == 2):
                            
                            #add 0 to count due target failed
                            count_target.append(0)
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal not reachable', end='', flush=True)
                            continue
                            
                        
                        #----------------------------------------------                       
                                                
                        elif(status == 3):
                            
                            #add 1 due target reached
                            count_target.append(1)
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal reached', end='', flush=True)
                            
                             
                             
                            print('Exiting')
                            print(*count_target, sep = ", ") 
                            
                            time.sleep(4)
                            # planning again
                            move_base_show()
                        
                            return 1
                            
                             #----------------------------------------------

                                
                        
                        elif(status == 4):
                            
                            #add 0 due target aborted
                            
                            count_target.append(0)
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal Cancelled - No plan avaible', end='', flush=True)
                            
                            
                            print('Exiting')
                            
                            
                            # planning again
                            move_base_show()
                            publisher_cancel = rospy.Publisher("/move_base/cancel", GoalID, queue_size = 1)
                            cancel = GoalID()
                            publisher_cancel.publish(cancel)
                            
                            time.sleep(4)
                            return 1
                            
                            #-----------------------------------------------------------------
                        
                        rate.sleep()

 
move_base()

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

ROSSerializationException: <class 'struct.error'>: 'required argument is not a float' when writing 'header: 
  seq: 1
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
goal_id: 
  stamp: 
    secs: 0
    nsecs:         0
  id: ''
goal: 
  target_pose: 
    header: 
      seq: 0
      stamp: 
        secs: 0
        nsecs:         0
      frame_id: "map"
    pose: 
      position: 
        x: ''
        y: ''
        z: 0.0
      orientation: 
        x: 0.0
        y: 0.0
        z: 0.0
        w: 1'